# 2D MENT

In [ ]:
import os
import sys

import numpy as np
import ot
import proplot as pplt
import scipy.interpolate
import torch
from tqdm.notebook import tqdm

import mentflow as mf
from mentflow.utils import grab

# local
import plotting 
import utils

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

## Data

In [ ]:
device = torch.device("cpu")
precision = torch.float32

def send(x):
    return x.type(precision).to(device)

In [ ]:
data_name = "spirals"
data_size = int(1.00e+06)
data_noise = None
xmax = 3.25
n_bins = 75
n_meas = 6

dist = mf.data.toy.gen_dist(data_name, noise=data_noise)
x0 = dist.sample(data_size)
x0 = send(x0)

angles = np.linspace(0.0, np.pi, n_meas, endpoint=False)
transfer_matrices = []
for angle in angles:
    matrix = mf.transform.rotation_matrix(angle)
    matrix = send(matrix)
    transfer_matrices.append(matrix)

transforms = []
for matrix in transfer_matrices:
    transform = mf.transform.Linear(matrix)
    transform = transform.to(device)
    transforms.append(transform)

bin_edges = torch.linspace(-xmax, xmax, n_bins + 1)
diagnostic = mf.diagnostics.Histogram1D(axis=0, bin_edges=bin_edges, kde=False)
diagnostic = diagnostic.to(device)

measurements = [diagnostic(transform(x0)) for transform in transforms]

## Model

In [ ]:
# prior = mf.models.ment.UniformPrior(d=2, scale=20.0, device=device)
prior = mf.models.ment.GaussianPrior(d=2, scale=2.0, device=device)

sampler_limits = 2 * [(-xmax, +xmax)]
sampler_limits = 1.1 * np.array(sampler_limits)
sampler = mf.sample.GridSampler(limits=sampler_limits, res=200, device=device)

model = mf.models.ment.MENT(
    d=2,
    transforms=transforms,
    measurements=measurements,
    diagnostic=diagnostic,
    prior=prior,
    sampler=sampler,
    interpolate="pchip",  # {"nearest", "linear", "pchip"}
    device=device,
)

## Training

In [ ]:
# Settings
n_iterations = 10
method = "integrate"  # {"integrate", "sample"}
method_kws = {
    "limits": [(-xmax, +xmax)],  # integration limits
    "shape": (150,),  # integration grid shape (resolution)
    "n": int(1.00e+05),  # number of particles (if method="sample")
}

vis_n_bins = 100
vis_n_samples = int(1.00e+05)
vis_res = 200

for iteration in range(n_iterations):
    # Evaluate the density on a grid.
    res = vis_res
    grid_coords = 2 * [torch.linspace(-xmax, xmax, vis_res)]
    grid_points = mf.utils.get_grid_points_torch(grid_coords)
    grid_points = send(grid_points)
    prob = model.prob(grid_points)
    prob = prob.reshape(vis_res, vis_res)
    prob = grab(prob)
    
    # Draw samples from the model and true distribution.
    x0 = dist.sample(vis_n_samples)
    x0 = grab(x0)
    x = model.sample(vis_n_samples)
    x = grab(x)

    # Simulate measurements
    predictions = model.simulate(method=method, **method_kws)

    # Plot the true samples, model samples, and model density.
    fig, axs = plotting.plot_dist(
        x0,
        x,
        prob=prob,
        coords=[grab(c) for c in grid_coords],
        n_bins=vis_n_bins,
        limits=(2 * [(-xmax, xmax)]),
    )
    pplt.show()

    # Plot overlayed simulated/measured projections.    
    fig, axs = plotting.plot_proj(
        [grab(measurement) for measurement in measurements],
        [grab(prediction) for prediction in predictions],
        bin_edges=grab(diagnostic.bin_edges),
        maxcols=7,
        kind="line",
        height=1.25
    )
    pplt.show()

    # Compute simulation-measurement discrepancy
    discrepancy = sum(
        torch.sum(torch.abs(predictions[i] - measurements[i]))
        for i in range(n_meas)
    )
    discrepancy = discrepancy / n_meas

    # Compute sliced wasserstein distance between true and model samples.
    distance = ot.sliced.sliced_wasserstein_distance(
        x0, x, n_projections=100, p=2, projections=None
    )

    # Print summary
    print("iteration = {}".format(iteration))
    print("D(y_model, y_true) = {}".format(discrepancy))
    print("SWD(x_model, x_true) = {}".format(distance))
    
    # Update the lagrange functions.
    model.gauss_seidel_iterate(method=method, **method_kws)